$$
\newcommand{\mat}[1]{\boldsymbol {#1}}
\newcommand{\mattr}[1]{\boldsymbol {#1}^\top}
\newcommand{\matinv}[1]{\boldsymbol {#1}^{-1}}
\newcommand{\vec}[1]{\boldsymbol {#1}}
\newcommand{\vectr}[1]{\boldsymbol {#1}^\top}
\newcommand{\rvar}[1]{\mathrm {#1}}
\newcommand{\rvec}[1]{\boldsymbol{\mathrm{#1}}}
\newcommand{\diag}{\mathop{\mathrm {diag}}}
\newcommand{\set}[1]{\mathbb {#1}}
\newcommand{\cset}[1]{\mathcal{#1}}
\newcommand{\norm}[1]{\left\lVert#1\right\rVert}
\newcommand{\pderiv}[2]{\frac{\partial #1}{\partial #2}}
\newcommand{\bb}[1]{\boldsymbol{#1}}
\newcommand{\E}[2][]{\mathbb{E}_{#1}\left[#2\right]}
\newcommand{\ip}[3]{\left<#1,#2\right>_{#3}}
\newcommand{\given}[]{\,\middle\vert\,}
\newcommand{\DKL}[2]{\cset{D}_{\text{KL}}\left(#1\,\Vert\, #2\right)}
\newcommand{\grad}[]{\nabla}
\newcommand{\norm}[1]{\left\lVert#1\right\rVert}
$$

# Part 2: Summary Questions
<a id=part2></a>

This section contains summary questions about various topics from the course material.

You can add your answers in new cells below the questions.

**Notes**

- Clearly mark where your answer begins, e.g. write "**Answer:**" in the beginning of your cell.
- Provide a full explanation, even if the question doesn't explicitly state so. We will reduce points for partial explanations!
- This notebook should be runnable from start to end without any errors.

### CNNs

1. Explain the meaning of the term "receptive field" in the context of CNNs.

**Answer:**
In the context of CNN the receptive field is the region of the input space that affects a particular unit of the network. 
For example when an image is proccessed the receptive field of a feature would be the amount of pixels in the input that takes place in the forward calculation.

    

2. Explain and elaborate about three different ways to control the rate at which the receptive field grows from layer to layer. Compare them to each other in terms of how they combine input features.

**Answer:**

1.Changing the <u>**kernel size**</u> is a way of controlling the rate at which the receptive field grows from layer to layer.
Changing it's value will increase or decrease the number of adjacent elements of the input that take place in the calculation of the output.

2.Using <u>**stride**</u> larger than 1 (sometimes stride=1 reffers to non stride convolution) will also control the receptive field. It will take place into the calculation of the output but opposed to changing the kernel size the stride will take into calculation of more distant elements.

3.Using <u>**dilated convolution**</u> which is like any regular convolution but with **gaps** in the input. That method would result in increasing receptive view of the network.
It differs than than the previous method because that dilated convolution does not involve **all of the input** in its calculations.

3. Imagine a CNN with three convolutional layers, defined as follows:

In [8]:
import torch
import torch.nn as nn

cnn = nn.Sequential(
    nn.Conv2d(in_channels=3, out_channels=4, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(in_channels=4, out_channels=16, kernel_size=5, stride=2, padding=2),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(in_channels=16, out_channels=32, kernel_size=7, dilation=2, padding=3),
    nn.ReLU(),
)

cnn(torch.rand(size=(1, 3, 1024, 1024), dtype=torch.float32)).shape

torch.Size([1, 32, 122, 122])

What is the size (spatial extent) of the receptive field of each "pixel" in the output tensor?

**Answer:**

We can calculate the receptive field of the Nth convolution layer in a network that contain <u>only convolutions</u> using this formula:

$$\sum_{i=1}^{N} [(k_i - 1) \cdot \prod_{j=1}^{i-1} s_i] + 1$$

where **k** is kernel size and **s** is stride.
In order to use that formula we will convert our given network to a network that contain only convolutions.

* activations maintain the receptive field so might be ignored.
* Pooling layer with kernel=2 could be replaced by convolution with kernel=2 and stride=2.
* Dilation can be converted to a much larger kernel size convolution. So Dilation of 2 with kernel=7 converts to kernel=$2\cdot(7-1)+1=13$.

**The converted network**

1. conv(k=3, s=1)
2. conv(k=2, s=2) used to be maxpool(2)
3. conv(k=5, s=2)
4. conv(k=2, s=2) used to be maxpool(2)
5. conv(k=17)

The size of the receptive field of the last layer is **112**.

$(3-1) + (2-1)\cdot1 + (5-1)\cdot2\cdot1 + (2-1)\cdot2\cdot2\cdot1 + (13-1)\cdot2\cdot2\cdot2\cdot1 + 1 = 112$

4. You have trained a CNN, where each layer $l$ is represented by the mapping $\vec{y}_l=f_l(\vec{x};\vec{\theta}_l)$, and $f_l(\cdot;\vec{\theta}_l)$ is a convolutional layer (not including the activation function).

  After hearing that residual networks can be made much deeper, you decide to change each layer in your network you used the following residual mapping instead $\vec{y}_l=f_l(\vec{x};\vec{\theta}_l)+\vec{x}$, and re-train.

  However, to your surprise, by visualizing the learned filters $\vec{\theta}_l$ you observe that the original network and the residual network produce completely different filters. Explain the reason for this.

**Answer:**

non-residual layer filters are being used in computation of the whole output based on the input while <u>residual</u> layer filters used in the computation of the difference betweem input and output of the layer. Therefore the filters are totally different.

### Dropout

1. Consider the following neural network:

In [2]:
import torch.nn as nn

p1, p2 = 0.1, 0.2
nn.Sequential(
    nn.Conv2d(in_channels=3, out_channels=4, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.Dropout(p=p1),
    nn.Dropout(p=p2),
)

Sequential(
  (0): Conv2d(3, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU()
  (2): Dropout(p=0.1, inplace=False)
  (3): Dropout(p=0.2, inplace=False)
)

If we want to replace the two consecutive dropout layers with a single one defined as follows:
```python
nn.Dropout(p=q)
```
what would the value of `q` need to be? Write an expression for `q` in terms of `p1` and `p2`.

**Answer:**

$q=1-(1-p_1)\cdot(1-p2)=1-(0.9\cdot0.8)=0.28$

Assume our input is $X$ units so after the first dropout of $p_1$ we are left with $x'=(1-p_1)\cdot X$ input units.
After the second dropout of $p_2$ we are left with $x''=(1-p_2)\cdot X'$ input units.

$X''=(1-p_2)\cdot (1-p_1)$

So we have lost $X-X''= X -(1-p_2)\cdot (1-p_1)\cdot X = [1 -(1-p_2)\cdot (1-p_1)]\cdot X$.
In order to replace the dropouts we will have to choose a $q$ that will lead to that amount of loss.
Therefore we can replace the two dropouts with 1 dropout of $q=1 -(1-p_2)\cdot (1-p_1)$.

2. **True or false**: dropout must be placed only after the activation function.

**Answer:**

**False**, for example in the case above the Dropout can be be placed <u>before or after</u> the ReLU and it will lead to the same results.
Assume our input $X = [2,2,2,-2,-2,-2]$ 

1. lets use the Dropout($p=0.5$) out before the ReLU:
$Dropout(X) = [0,2,0,-2,0,-2] = X'$

    Assume the Relu is max between 0,x
    
    $ReLU(X') = [0,2,0,0,0,0] = X''$
    
2. This time we will use the Relu first and then the Dropout and achieve the same results.
$Relu(X) = [2,2,2,0,0,0] = X'$

    Assume the Dropout chose the **same indexes as before**:

    $Dropout(X') = [0,2,0,0,0,0] = X''$
    
In colnclusion we can see that it doesnt matter if the Dropout is placed before or after the activation.



3. After applying dropout with a drop-probability of $p$, the activations are scaled by $1/(1-p)$. Prove that this scaling is required in order to maintain the value of each activation unchanged in expectation.

**Answer:**

With no scaling, the expected output after $Dropout$ of input $X$ is $(1-p)\cdot X$. In order to maintain the value of each activation unchanged in expectation we scale the output during training by scaling the activation by $1/(1-p)$ which leads to the expected output:

$(1-p)\cdot X / (1-p) = X$

### Losses and Activation functions

1. You're training a an image classifier that, given an image, needs to classify it as either a dog (output 0) or a hotdog (output 1). Would you train this model with an L2 loss? if so, why? if not, demonstrate with a numerical example. What would you use instead?

**Answer:**

We would <u>NOT</u> use L2 loss for this task because this is a **classifier** task and the L2 loss is preferable for regressions.
We would prefer to use a <u>binary cross entropy</u> for the classifying problem as we have seen in class that it fits better to classifying due to penalizing and rewarding predictions better.

Let's see why L2 will not achieve what we want.
Examine the case we have 2 samples of dog, $x_1,x_2$ and our model predicted their probabilities: $p_1,p_2 = 0.45,0.55$.

$L2(p_1) = (1-p_1)^2 = 0.3025$

$L2(p_1) = (1-p_2)^2 = 0.2025$

As we can see the values are not that far from each other, the difference between them is 1.

lets examine another case of 2 samples which are dogs with probabilities $p_1,p_2 = 0.32, 0.4$

$L2(p_1) = (1-p_1)^2 = 0.4624$

$L2(p_1) = (1-p_2)^2 = 0.36$

As we can see the values are **ALSO** not that far from each other, the difference between them is ~1 **BUT** this time both classified with the same class as opposed to the former scenario.


2. After months of research into the origins of climate change, you observe the following result:

<center><img src="https://sparrowism.soc.srcf.net/home/piratesarecool4.gif" /></center>

You decide to train a cutting-edge deep neural network regression model, that will predict the global temperature based on the population of pirates in `N` locations around the globe.
You define your model as follows:

In [1]:
import torch.nn as nn

N = 42  # number of known global pirate hot spots
H = 128
mlpirate = nn.Sequential(
    nn.Linear(in_features=N, out_features=H),
    nn.Sigmoid(),
    *[
        nn.Linear(in_features=H, out_features=H), nn.Sigmoid(),
    ]*24,
    nn.Linear(in_features=H, out_features=1),
)

While training your model you notice that the loss reaches a plateau after only a few iterations.
It seems that your model is no longer training.
What is the most likely cause?

**Answer:**

The most likely cause for that to happen is the <u>vanishing gradient problem</u>.
The problem is that the gradient becomes very small and therefore preventing the weight from changing its value. In the worst case, this may completely stop the neural network from further training.

We think that in the chosen model the abuse of Linear and sigmoid layers causing the gradient to vanish because that the sigmoid maps an input space to small output space which will result in small gradients.

3. Referring to question 2 above: A friend suggests that if you replace the `sigmoid` activations with `tanh`, it will solve your problem. Is he correct? Explain why or why not.

**Answer:**

Using tanh would'nt solve our problem, because that tanh also suffers from the same known problem.
Like the sigmoid which maps values to the range of $[0,1]$, tanh maps to the range of $[-1,1]$.
This behaviour causing large inputs to be mapped to small values resulting in vanishing gradient.

4. Regarding the ReLU activation, state whether the following sentences are **true or false** and explain:
  1. In a model using exclusively ReLU activations, there can be no vanishing gradients.
  1. The gradient of ReLU is linear with its input when the input is positive.
  1. ReLU can cause "dead" neurons, i.e. activations that remain at a constant value of zero.

**Answer:**

A. **False**, ReLU activations can suffer from vanishing gradients in the case that some layers 

B. **False**, the gradient of ReLU for a positive input is constant and equals to 1, Therefore the the gradient will not change proportionaly to the input.

C. **True**, ReLU can cause "dead" neurons for example in the case that a weight update from a gradient would result a continous negative output for some neurons causing those neurons to be "dead".
Those neurons "die" becuase that their gradient will be 0, which prevent weights updating therefore the neuron will stay "dead".

### Optimization

1. Explain the difference between: stochastic gradient descent (SGD), mini-batch SGD and regular gradient descent (GD).

**Answer:**

1. All of them updates set of parameters iteratively in order to minimize an error function.
They differ by the way they iterate.

<u>Gradient Descent (GD)</u> - iterates over <u>all</u> of the samples in the training set in order to perform a single update.

<u>Stochastic Gradient Descent (SGD)</u> - going through one sample per each update.

<u>Mini-Batch Stochastic Gradient Descent</u> - using the same method as SGD but with a small batch of samples.

Those differences are easily noticeable in the case of large input, GD could be not practical to use as we have to iterate over <u>all samples</u> every update. Therefore the SGD'S would be much more practicle and perform faster.

2. Regarding SGD and GD:
  1. Provide at least two reasons for why SGD is used more often in practice compared to GD.
  2. In what cases can GD not be used at all?

**Answer:**

A. Firstly, in the case of a large amount of samples GD would have to run through all of them for a single update.GD would use much more memory vs. the SGD (which needs 1 sample or a batch).
Secondly the SGD method converages much faster than the GD for large amount of samples and therefore used more often in practice.

B. In the case of a huge dataset, GD won't be able to load all the samples to the memory and even if it did it would converage very slowly.

3. You have trained a deep resnet to obtain SoTA results on ImageNet.
While training using mini-batch SGD with a batch size of $B$, you noticed that your model converged to a loss value of $l_0$ within $n$ iterations (batches across all epochs) on average.
Thanks to your amazing results, you secure funding for a new high-powered server with GPUs containing twice the amount of RAM.
You're now considering to increase the mini-batch size from $B$ to $2B$.
Would you expect the number of of iterations required to converge to $l_0$ to decrease or increase when using the new batch size? explain in detail.

**Answer:**

We would expect the number of iterations required to converge to $l_0$ to **decrease**.
Eventhough we increased the batch size from **B** to **2B** we expect the SGD to converge in less iterations because of that each iteration will last longer but supply better error gradients (more samples in the batch will increase accuracy).
Our new, more precise error gradients will result in a convergence in less iterations.
Comment: less iteration **DOES NOT** mean less time.

4. For each of the following statements, state whether they're **true or false** and explain why.
  1. When training a neural network with SGD, every epoch we perform an optimization step for each sample in our dataset.
  1. Gradients obtained with SGD have less variance and lead to quicker convergence compared to GD.
  1. SGD is less likely to get stuck in local minima, compared to GD.
  1. Training  with SGD requires more memory than with GD.
  1. Assuming appropriate learning rates, SGD is guaranteed to converge to a local minimum, while GD is guaranteed to converge to the global minimum.
  1. Given a loss surface with a narrow ravine (high curvature in one direction): SGD with momentum will converge more quickly than Newton's method which doesn't have momentum.

**Answer:**

A. **True**, every epoch we perform an optimization step for <u>each sample in our dataset</u> using one sample or a batch if it's a mini-batch SGD.

B. **False**, Gradients obtained with SGD have <u>larger</u> variance compared to GD because it takes into consideration only a small subset on each optimization step. That larger variance contributes to convergence.

C. **True**, The fact that the SGD's gradient has more variance and it's randomness, make this method more likely to escape of such cases.

D. **False**, SGD requires <u>less</u> memory then the GD.
SGD uses one sample or a small batch for its optimization step while GD going through the whole dataset which leads to heavier load on the memory because we have to fit more samples into it and at same time.

E. **False**, in <u>convex loss surfaces</u> the local and global minimum are guaranteed, but for <u>non convex surfaces</u> only a local minimum can be guaranteed for both of them.

F. **True**, both of them will probably converge, but the calculation of second derivitives in Newton's method will probably result in higher computation times which will lead to a slower converge then the SGD momentum (which will converge faster).

5. In tutorial 5 we saw an example of bi-level optimization in the context of deep learning, by embedding an optimization problem as a layer in the network.
  **True or false**: In order to train such a network, the inner optimization problem must be solved with a descent based method (such as SGD, LBFGS, etc).
  Provide a mathematical justification for your answer.

**Answer:**

**False**, The inner optimization problem can be solved as a constraint in the network's loss function. Therefore we can minimize an expression $L = f(x) + \lambda g(x)$ without using a descent based method just like we find a minimum on a regular function.

6. You have trained a neural network, where each layer $l$ is represented by the mapping $\vec{y}_l=f_l(\vec{x};\vec{\theta}_l)$ for some arbitrary parametrized functions $f_l(\cdot;\vec{\theta}_l)$.
  Unfortunately while trying to break the record for the world's deepest network, you discover that you are unable to train your network with more than $L$ layers.
  1. Explain the concepts of "vanishing gradients", and "exploding gradients".
  2. How can each of these problems be caused by increased depth?
  3. Provide a numerical example demonstrating each.
  4. Assuming your problem is either of these, how can you tell which of them it is without looking at the gradient tensor(s)?

**Answer:**

A. Both concepts causes the network to be untrainable.

<u>Vanishing gradients</u> - a phenomenon when the gradients of the loss function are getting very small and close to zero which prevents the weights from updating properly.

<u>Exploding gradients</u> - a phenomenon when the gradients of the loss function are getting very large, also leading to a poor update of the weights which results in unstable and untrainable network.

B. Following the chain rule, calculating the gradient of the loss requires multiplying the derivatives of all layers.
The multiplying causes and exponential behaviour:

<u>large values</u> - in a deep network results in exponential growth of the gradients and causes the **exploding gradients**

<u>low values</u> - in a deep network results in exponential decrease of the gradients and causes the **vanishing gradients**

C. an example for the <u>vanishing gradients</u>:

lets assume an input of size 16 and 1 sigmoid functions:
After the first application of $sigmoid(16)=~0.99999..$, and it's deriviative is $1.125x10^{-7}$.
As we can witness we are already very close to 0, if we will keep with a deep network we will reach vanishing gradients in no time.

an example for the <u>exploding gradients</u>:

lets assume an input of size 516 and activation functions $f(x)=x^4$:
After the first activation we get a derivative of $\frac{d}{dx} = 4x^3 = 536870912$ which is already huge.
going deeper in the network will increase that value exponentially fast and we will achieve exploding gradients.

D. You can tell by the loss function:

<u>constant value of loss</u> - we will notice that the loss being constant and weights not updating their values which are side effects of the <u>vanishing gradients</u>

<u>unstable value of loss</u> - we will notice that the loss being unstable and weights getting very large in value, maybe even overflow which are side effects of the <u>exploding gradients</u>.


### Backpropagation

1. You wish to train the following 2-layer MLP for a binary classification task:
  $$
  \hat{y} =\mat{W}_2~ \varphi(\mat{W}_1 \vec{x}+ \vec{b}_1) + \vec{b}_2
  $$
  Your wish to minimize the in-sample loss function is defined as
  $$
  L_{\mathcal{S}} = \frac{1}{N}\sum_{i=1}^{N}\ell(y,\hat{y}) + \frac{\lambda}{2}\left(\norm{\mat{W}_1}_F^2 + \norm{\mat{W}_2}_F^2 \right)
  $$
  Where the pointwise loss is binary cross-entropy:
  $$
  \ell(y, \hat{y}) =  - y \log(\hat{y}) - (1-y) \log(1-\hat{y})
  $$
  
  Write an analytic expression for the derivative of the final loss $L_{\mathcal{S}}$ w.r.t. each of the following tensors: $\mat{W}_1$, $\mat{W}_2$, $\mat{b}_1$, $\mat{b}_2$, $\mat{x}$.

**Answer**:

$$
\ell(y, \hat{y}) =  - y \log(\hat{y}) - (1-y) \log(1-\hat{y})
$$  

$$
\frac{dL_S}{d{W}_1} = \frac{1}{N}\sum_{i=1}^{N}\frac{d\ell}{d{W}_1} + \lambda(\norm{\mat{W}_1}_F = \frac{1}{N}\sum_{i=1}^{N}\frac{d\ell}{d\hat{y}}\frac{d\hat{y}}{d\mat{W}_1} + \lambda(\norm{\mat{W}_1}_F
$$

$$
\frac{dL_S}{d{W}_2} = \frac{1}{N}\sum_{i=1}^{N}\frac{d\ell}{d{W}_2} + \lambda(\norm{\mat{W}_2}_F = \frac{1}{N}\sum_{i=1}^{N}\frac{d\ell}{d\hat{y}}\frac{d\hat{y}}{d\mat{W}_2} + \lambda(\norm{\mat{W}_2}_F
$$

$$
\frac{dL_S}{d\vec{b}_1} = \frac{1}{N}\sum_{i=1}^{N}\frac{d\ell}{d\vec{b}_1} = \frac{1}{N}\sum_{i=1}^{N}\frac{d\ell}{d\hat{y}}\frac{d\hat{y}}{d\vec{b}_1} 
$$

$$
\frac{dL_S}{d\vec{b}_2} = \frac{1}{N}\sum_{i=1}^{N}\frac{d\ell}{d\vec{b}_2} = \frac{1}{N}\sum_{i=1}^{N}\frac{d\ell}{d\hat{y}}\frac{d\hat{y}}{d\vec{b}_2} 
$$

$$
\frac{dL_S}{d\vec{x}} = \frac{1}{N}\sum_{i=1}^{N}\frac{d\ell}{d\vec{x}} = \frac{1}{N}\sum_{i=1}^{N}\frac{d\ell}{d\hat{y}}\frac{d\hat{y}}{d\vec{x}} 
$$

$$
\frac{d\ell}{d\hat{y}} = -\frac{y}{\hat{y}} +\frac{(1-y)}{1-\hat{y}} 
$$

$$
\vec{z} = \mat{W}_1 \vec{x}+ \vec{b}_1
$$

$$
\frac{d\hat{y}}{d\mat{W}_1} = {W}_2\frac{d\varphi}{d\vec{z}}\vec{x}
$$

$$
\frac{d\hat{y}}{d\mat{W}_2} = \varphi(\mat{W}_1 \vec{x}+ \vec{b}_1)
$$

$$
\frac{d\hat{y}}{d\vec{b}_1} = {W}_2\frac{d\varphi}{d\vec{z}}
$$

$$
\frac{d\hat{y}}{d\vec{b}_2} = 1
$$

$$
\frac{d\hat{y}}{d\vec{x}} = {W}_2\frac{d\varphi}{d\vec{z}}\mat{W}_1
$$

2. Derivative a function $f(\vec{x})$ at a point $\vec{x}_0$ is
  $$
  f'(\vec{x}_0)=\lim_{\Delta\vec{x}\to 0} \frac{f(\vec{x}_0+\Delta\vec{x})-f(\vec{x}_0)}{\Delta\vec{x}}
  $$
  
  1. Explain how this formula can be used in order to compute gradients of neural network parameters numerically, without automatic differentiation (AD).
  
  2. What are the drawbacks of this approach? List at least two drawbacks compared to AD.

A. Numerically that formula can be used to compute gradients of neural network parameter without automatic differentiation that way:

1. Choose a small value, lets say $\epsilon=0.00000001$
2. Compute the values $f(\vec{x}_0+\hat\epsilon), f(\vec{x}_0)$ (adding each value of the vector the value of epsilon).
3. Assign the values to the formula to get the gradient of $f(\vec{x})$ at point $\vec{x}_0$

B. The drawbacks of this approach are:

* <u>Not accurate</u> - the choice of $\Delta\vec{x}$ is crucial.
    choosing a value too big the approximation of the deriviative is poor, and choosing va value too small would cause an error because of floating point values being cut off which will lead to inaccurate gradient values.
    
* <u>Stability</u> - Numericly this operation is not stable when computing for large amount of parameters because we are summing a large value to a small value and dividing by a small value.
    as we learny in numerical algorithems this may cause to instability.
    
* <u>Long computation time</u> - in order to compute a gradient of function with $n$ parameters we have to make $O(n)$ operations.
    Given a network a large number of parameters this computation will consume a lot of resources.

3. Given the following code snippet:
  1. Write a short snippet that implements that calculates gradient of `loss` w.r.t. `W` and `b` using the approach of numerical gradients from the previous question.
  2. Calculate the same derivatives with autograd.
  3. Show, by calling `torch.allclose()` that your numerical gradient is close to autograd's gradient.

In [10]:
import torch

N, d = 100, 5
dtype = torch.float64
X = torch.rand(N, d, dtype=dtype)
W, b = torch.rand(d, d, requires_grad=True, dtype=dtype), torch.rand(d, requires_grad=True, dtype=dtype)

def foo(W, b):
    return torch.mean(X @ W + b)

loss = foo(W, b)
print(f"{loss=}")

# TODO: Calculate gradients numerically for W and b
delta_x = 10**-8
w_gradient, b_gradient = torch.zeros_like(W), torch.zeros_like(b)

# calculating b_gradient
for i in range(0,d):
    updated_b = torch.clone(b)
    updated_b[i] += delta_x
    b_gradient[i] = (foo(W,updated_b) - foo(W,b)) / (delta_x)
    
# calculating w_gradient
for i in range(0,d):
    for j in range(0,d):
            updated_w = torch.clone(W)
            updated_w[i,j] += delta_x
            w_gradient[i,j] = (foo(updated_w,b) - foo(W,b)) / (delta_x)
            

# TODO: Compare with autograd using torch.allclose()
loss.backward()
autograd_W = W.grad
autograd_b = b.grad
assert torch.allclose(w_gradient, autograd_W)
assert torch.allclose(b_gradient, autograd_b)

loss=tensor(1.9242, dtype=torch.float64, grad_fn=<MeanBackward0>)


### Sequence models

1. Regarding word embeddings:
  1. Explain this term and why it's used in the context of a language model.
  1. Can a language model like the sentiment analysis example from the tutorials be trained without an embedding? If yes, what would be the consequence for the trained model? if no, why not?

**Answer:**

A. Word embeddings are a type of a word representation, a mapping between token index to a higher dimension representation.
The higher dimension space separates the words by their semantics and caegories. The separation occurs during the training.

B. It won't work well as <u>with</u> embedding.
The model will learn a classification by 1 feature only like using a 1 hot representation where every word is a vector with only one 1 and the rest are zeros, it fails to take into consideration the semantics of the word. 
Embedding take the semantic into consideration and the weights will be more similar in similar words.


2. Considering the following snippet, explain:
  1. What does `Y` contain? why this output shape?
  2. How you would implement `nn.Embedding` yourself using only torch tensors. 

In [11]:
import torch.nn as nn

X = torch.randint(low=0, high=42, size=(5, 6, 7, 8))
embedding = nn.Embedding(num_embeddings=42, embedding_dim=42000)
Y = embedding(X)
print(f"{Y.shape=}")

Y.shape=torch.Size([5, 6, 7, 8, 42000])


**Answer:**

A. Y contains a mapping of each sample of X to a 42000 vector.
Each of X's values are at the range of $[0,42]$ and will be represented by a 42000 vector.
It is because the embedding dimension is 42000 and we can see that the first 4 dimensions of Y are the same as X.

B. Simple implementation is using nn.Linear with 1-hot representation.
Let's assume we have a 420 words, and we would like a 42 dimension embedding.

We can simply use a nn.Linear(420,42) for that case, and each word will be represented with 419 zeros and one 1.

3. Regarding truncated backpropagation through time (TBPTT) with a sequence length of S: State whether the following sentences are **true or false**, and explain.
  1. TBPTT uses a modified version of the backpropagation algorithm.
  2. To implement TBPTT we only need to limit the length of the sequence provided to the model to length S.
  3. TBPTT allows the model to learn relations between input that are at most S timesteps apart.

**Answer:**

1. **True**, TBPTT uses a modified version of backpropagation algorithm where the weights are updated using deriviatives <u>only</u> from the last $K$ samples unlike regular BP which derivate through all time stamps.

2. **False**, in order to implement TBPTT we need to limit both length of backpropagation sequence <u>and</u> the length of the forward sequence.

3. **True**,  the length of the forward pass is $S$, and there is a hidden state which is transffered between each time stamp in S and will provide relation to the previous time stamps.

### Attention

1. In tutorial 7 (part 2) we learned how to use attention to perform alignment between a source and target sequence in machine translation.
  1. Explain qualitatively what the addition of the attention mechanism between the encoder and decoder does to the hidden states that the encoder and decoder each learn to generate (for their language). How are these hidden states different from the model without attention
  
  2. After learning that self-attention is gaining popularity thanks to the shiny new transformer models, you decide to change the model from the tutorial: instead of the queries being equal to the decoder hidden states, you use self-attention, so that the keys, queries and values are all equal to the encoder's hidden states (with learned projections). What influence do you expect this will have on the learned hidden states?


**Answer:**

A. <u>Without attention</u> - Hidden states from the encoder have to deliver all of the sequence information. The decoder hidden states will begin from the last hidden state of the encoder and continue with the decoder's previous state.

<u>With attention</u> - at each timestamp the decoder is searching the whole sequence latent space and chooses the best fitted word. The attention mechanism is acting like a feedback of the decoder's hidden state.
The hidden states of the encoder should translate to a single word and not a sequence because the attention would be used in order to choose the best fit next word from the sequence.

B. We expect that the hidden states will genereate the next word with consideration of adjacent words and the projections would create a more related embedding of the sequence.

### Unsupervised learning

1. As we have seen, a variational autoencoder's loss is comprised of a reconstruction term and  a KL-divergence term. While training your VAE, you accidentally forgot to include the KL-divergence term.
What would be the qualitative effect of this on:

  1. Images reconstructed by the model during training ($x\to z \to x'$)?
  1. Images generated by the model ($z \to x'$)?

**Answer:**

The KL-divergence term acts like a regulizer, and measures the distance between probability distributions.

A. In that case we only use the data reconstruction loss so we expect the model to overfit to the data so the expected Images reconstructed should be similar to the input.

B. The model was trained without the KL-divergence term so the reconstructed images would be similar to the data the model "saw" at the training stage.

2. Regarding VAEs, state whether each of the following statements is **true or false**, and explain:
  1. The latent-space distribution generated by the model for a specific input image is $\mathcal{N}(\vec{0},\vec{I})$.
  2. Every time we feed an image to the encoder, then decode the result, we'll get the same reconstruction.
  3. Since the real VAE loss term is intractable, what we actually minimize instead is it's upper bound, in the hope that the bound is tight.

**Answer:**

A. **False**, The VAE sets a prior distibution to the latent space.
Our ability to map any distribution to any other helps us to fix the prior distribution to a normal $\mathcal{N}(\vec{0},\vec{I})$.

B. **False**, the reconstructed image would be different then others because that the latent space allows a random sampling from the normal distribution resulting in generation of different images.

C. **True**, we have seen in class that the actual VAE loss is intractable because the KL-divergence cannot be computed by the actual posterior because it needs the estimated evidence distribution. We minimize the ELBO instead, and hoping that that upper bound is tight.

2. Regarding GANs, state whether each of the following statements is **true or false**, and explain:
  1. Ideally, we want the generator's loss to be low, and the discriminator's loss to be high so that it's fooled well by the generator.
  2. It's crucial to backpropagate into the generator when training the discriminator.
  3. To generate a new image, we can sample a latent-space vector from $\mathcal{N}(\vec{0},\vec{I})$.
  4. It can be beneficial for training the generator if the discriminator is trained for a few epochs first, so that it's output isn't arbitrary.
  5. If the generator is generating plausible images and the discriminator reaches a stable state where it has 50% accuracy (for both image types), training the generator more will further improve the generated images.

**Answer:**

A. **True**, ideally we would want the generator's loss to be low and the discriminator's loss to be high so that the generator generates good images and the discriminator gets fooled. <u>but</u> keep in mind that those loss values not necessarily mean good generation that fools the discriminator. it could be that the discriminator's performance is bad for some other reason and not that the images are so good it fools it.

B. **False**, we dont have to backpropagate into the generator while training the discriminator because the training of the discriminator update consists of updating the weights through BP from the discriminator's loss through it's own network.

C. **True**, this is exactly what we have done in hw3 with GAN.
The generator is trained to map from a latent space distribution to an image.

D. **True**, training the discriminator for a few epochs first will contribute to the training proccess because it will be ahead of the generator and will be able to easily seperate between fake and real images which will result in shorter training time.

E. **False**, in the case that the discriminator have reached 50% accuracy, the generator has no useful information to get from the discriminator on how to improve it's generations. if the discriminator is working properly the case might be that the generator achieved so good results that the discriminator can no longer notice the difference between fake and real.
Therefore the generated images can not be improved.

### Graph Neural Networks

1. You have implemented a graph convolutional layer based on the following formula, for a graph with $N$ nodes:
$$
\mat{Y}=\varphi\left( \sum_{k=1}^{q} \mat{\Delta}^k \mat{X} \mat{\alpha}_k + \vec{b} \right).
$$
  1. Assuming $\mat{X}$ is the input feature matrix of shape $(N, M)$: what does $\mat{Y}$ contain in it's rows?
  1. Unfortunately, due to a bug in your calculation of the Laplacian matrix, you accidentally zeroed the row and column $i=j=5$ (assume more than 5 nodes in the graph).
What would be the effect of this bug on the output of your layer, $\mat{Y}$?

**Answer:**

A. The rows of **Y** contains the output feature map from the layer for all the nodes in the graph. 

B. due to the bug the 5th row in the output **Y** will be a constant 0, which will make the output feature map absolete, also the weights represented by the laplacian matrix for the input features will ignore the input features for the 5th node.
This will change the computation because the 5th node is not counted in the average but still exists.

2. We have discussed the notion of a Receptive Field in the context of a CNN. How would you define a similar concept in the context of a GCN (i.e. a model comprised of multiple graph convolutional layers)?

**Answer:**

The definition for CNNs talks about the region of input features that may affect calculations.
in the GCN we can define similar definition only with input vertex features which affects the calculation of that feature in a forward pass.
This receptive field would change according to the graph geometry.